In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.models import Model
import cv2
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard, EarlyStopping
model_vgg16_conv = VGG16(weights='imagenet')
model_vgg16_conv.layers[:-3]
x = Dense(512, activation='relu')(model_vgg16_conv.layers[-2].output)
x = Dense(1, activation='sigmoid', name='predictions')(x)

#Then create the corresponding model 
my_model = Model(input=model_vgg16_conv.input, output=x)

my_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


C:\Users\nichil\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\nichil\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


In [30]:
my_model.output_shape



(None, 1)

In [2]:
for layer in my_model.layers[:22]:
    layer.trainable = False

In [3]:
train_data_dir = 'images/train'
validation_data_dir = 'images/validation'
test_data_dir='images/test'

filepath="Models-{epoch:02d}-{val_acc:.2f}"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [checkpoint]
#callbacks = [TensorBoard(), EarlyStopping(monitor='val_loss',patience)]
checkpoint_callback = ModelCheckpoint(filepath+'.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks = [checkpoint_callback, EarlyStopping(monitor='val_loss',patience=30)]

nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 32
img_height= 224
img_width = 224
train_datagen = ImageDataGenerator(
     rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen=ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

test_generator=test_datagen.flow_from_directory(test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')


Found 3970 images belonging to 2 classes.
Found 689 images belonging to 2 classes.
Found 213 images belonging to 2 classes.


In [ ]:
my_model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800// batch_size,
        callbacks=callbacks)

score =my_model.evaluate_generator(test_generator)
#score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
my_model.save('my_model.h5')
#model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
 1/62 [..............................] - ETA: 1:24:08 - loss: 0.6056 - acc: 0.9062

In [4]:
my_model.save('my_model.h5')
